In [5]:
# Import NetworkX to work with cycles
import networkx as nx

# Convert the Sage Graph to a NetworkX graph
G_sage = nx.Graph([(0, 1), (0, 2), (0, 3), (1, 2), (2, 3), (3, 1)])
G = G_sage.to_directed()  # Convert to a directed graph to find cycles

# Function to count cycles containing each edge
def count_cycles_for_each_edge(graph):
    # Find all simple cycles in the directed graph
    all_cycles = list(nx.simple_cycles(graph))
    
    # Dictionary to store the count of cycles for each edge
    edge_cycle_count = {}
    
    # Iterate over each edge in the original (undirected) graph
    for edge in graph.to_undirected().edges():
        # Count cycles containing the edge, excluding cycles of length 2
        cycle_count = sum(1 for cycle in all_cycles
                          if len(cycle) > 2 and  # Exclude cycles of length 2
                          edge[0] in cycle and edge[1] in cycle and
                          cycle.index(edge[0]) == (cycle.index(edge[1]) - 1) % len(cycle))
        
        # Store the count in the dictionary
        edge_cycle_count[edge] = cycle_count
    
    return edge_cycle_count

# Call the function and print the result
edge_cycle_counts = count_cycles_for_each_edge(G)
print("Cycle counts for each edge:")
for edge, count in edge_cycle_counts.items():
    print(f"Edge {edge}: {count} cycles")



Cycle counts for each edge:
Edge (0, 1): 4 cycles
Edge (0, 2): 4 cycles
Edge (0, 3): 4 cycles
Edge (1, 2): 4 cycles
Edge (1, 3): 4 cycles
Edge (2, 3): 4 cycles


In [9]:
# Import the necessary modules
from sage.graphs.graph_generators import graphs
import csv

# Generate non-isomorphic graphs with 9 vertices and 21 edges
graph_generator = graphs.nauty_geng("9 21")

# Convert the generator to a list to iterate over each graph
graphs_list = list(graph_generator)

# Step 1: Generate all possible edges for 9 vertices (0 to 8)
possible_edges = [(i, j) for i in range(9) for j in range(i + 1, 9)]

# Step 2: Create a dictionary mapping each edge to its index in the vector
edge_to_index = {edge: idx for idx, edge in enumerate(possible_edges)}

# List to store vectorized graphs
vectorized_graphs = []

# Function to count cycles containing a given edge (excluding length-2 cycles)
def count_cycles_with_edge(graph, target_edge):
    # Convert the graph to a NetworkX format for cycle enumeration
    nx_graph = graph.networkx_graph().to_directed()
    all_cycles = list(nx.simple_cycles(nx_graph))  # Find all simple cycles
    
    # Count cycles containing the target edge, excluding cycles of length 2
    count = sum(1 for cycle in all_cycles
                if len(cycle) > 2 and 
                target_edge[0] in cycle and target_edge[1] in cycle and 
                cycle.index(target_edge[0]) == (cycle.index(target_edge[1]) - 1) % len(cycle))
    return count

# Step 3: Vectorize each graph based on cycle counts
for G in graphs_list:
    # Initialize a zero-vector of length 36
    vectorized_graph = [0] * len(possible_edges)
    
    # Calculate the cycle count for each edge in the graph
    for edge in G.edges(labels=False):
        # Sort the edge to ensure it’s in (min, max) format
        canonical_edge = (min(edge[0], edge[1]), max(edge[0], edge[1]))
        
        # If the edge is in our list of possible edges, count cycles and store in vector
        if canonical_edge in edge_to_index:
            index = edge_to_index[canonical_edge]
            cycle_count = count_cycles_with_edge(G, canonical_edge)
            vectorized_graph[index] = cycle_count
    
    # Append the vectorized form to the list
    vectorized_graphs.append(vectorized_graph)

# Write the vectorized graphs to a CSV file
with open("vectorized_graphs1.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    # Optional: Write header (Column names like "Edge_1", "Edge_2", ...)
    writer.writerow([f"Edge_{i+1}" for i in range(len(possible_edges))])
    # Write each vectorized graph as a row
    writer.writerows(vectorized_graphs)
